In [54]:
import time
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import json
from key import API_KEY

In [57]:

def get_trains(dir, train_ids, queue):
    req = requests.get(f'http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key={API_KEY}&mapid={dir}')
    elem = ET.fromstring(req.text)
    TODAY = datetime.today()
    current_trains = []
    for i, train in enumerate(elem.findall('eta')):
        train_time = datetime.combine(TODAY, datetime.strptime(train.find('arrT').text.split(' ')[-1], '%H:%M:%S').time())
        train_id = train.find('rn').text
        current_trains.append(train_id)
        if train_id not in train_ids:
            train_ids.add(train_id)
            queue.setdefault(train_id, train_time)
        else:
            queue[train_id] = train_time

    to_be_removed = [id for id in train_ids if id not in current_trains]

    for id in to_be_removed:
        train_ids.remove(id)
        queue.pop(id)

    return train_ids, queue


def main():
    ohare_train_ids, fp_train_ids = set(), set()
    ohare_queue, fp_queue = {}, {}
    while True:
        ohare_train_ids, ohare_queue = get_trains('40670', ohare_train_ids, ohare_queue)
        fp_train_ids, fp_queue = get_trains('40220', fp_train_ids, fp_queue)
        next_ohare = min(ohare_queue, key=ohare_queue.get)
        next_fp = min(fp_queue, key=fp_queue.get)
        print(f"{next_ohare=} - {ohare_queue[next_ohare]}")
        print(f"{next_fp=} - {fp_queue[next_fp]}")
        time.sleep(5)

main()


next_ohare='118' - 2023-02-10 15:01:57
next_fp='218' - 2023-02-10 15:03:36
next_ohare='118' - 2023-02-10 15:01:57
next_fp='218' - 2023-02-10 15:03:36
next_ohare='118' - 2023-02-10 15:01:57
next_fp='218' - 2023-02-10 15:03:36
next_ohare='214' - 2023-02-10 15:04:14
next_fp='218' - 2023-02-10 15:04:17
next_ohare='214' - 2023-02-10 15:04:14
next_fp='218' - 2023-02-10 15:04:17
next_ohare='214' - 2023-02-10 15:04:14
next_fp='218' - 2023-02-10 15:04:17
next_ohare='214' - 2023-02-10 15:04:14
next_fp='218' - 2023-02-10 15:04:17


KeyboardInterrupt: 